In [1]:
import torch
import torch.nn as nn
import cv2
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
class FaceMaskDataset(Dataset):
    def __init__(self, img_path, targets, transform=None):
        super().__init__()
        self.img_path = img_path
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self, index):
        img_path = self.img_path[index]
        target = self.targets[index]
        image = np.array(Image.open(img_path).convert('RGB'))

        if self.transform:
            image = self.transform(image)

        return image, target

In [4]:
class Detector(nn.Module):
    def __init__(self):
        self.model = self.load_model().to(device)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def load_model(self):
        model = models.resnet50(pretrained=True)
        for param in model.parameters():
            param.requires_grad = True

        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)
        model.eval()

        return model
    
    def inference(self, frame):
        frame = self.transform(frame).unsqueeze(0).to(device)
        with torch.no_grad():
            out = self.model(frame)

        prob = torch.softmax(out, dim=1)[0]
        mask_prob = prob[1].item()
        no_mask_prob = prob[0].item()
        label = 'mask' if mask_prob > no_mask_prob else 'no mask'

        return label